## How to define transformers for variables and objectives

In [ ]:
# Import the variable, objectives, sampler, acquisition function, and the optimisation classes
from nemo_bo.opt.variables import ContinuousVariable, VariablesList
from nemo_bo.opt.objectives import RegressionObjective, ObjectivesList
from nemo_bo.acquisition_functions.expected_improvement.expected_improvement import (
    ExpectedImprovement,
)
from nemo_bo.opt.samplers import LatinHyperCubeSampling
from nemo_bo.opt.optimisation import Optimisation

### Specifying transformation types for variables and objectives

When no transformation type is specified for a variable or objective, it will default to the one specified by  each machine learning model type's `default_X_transform_type` attribute. When `transformer` is specified, it will override this default transformation type and use the method that is passed. Each variable and objective is transformed separately from each other.

| Transformation type | String | Description |
| --- | --- | --- |
| Min-max normalisation | "normalisation" | Scales the data to between 0 and 1 using the specified lower and upper bounds |
| Standardisation | "standardisation" | Scales the data to have a mean of 0 and a standard deviation of 1 |
| *None* | "none" | No transformation is applied |
| *User custom method* | - | Instances of transformers can be used, as long as they have fit_transform, transform, and inverse_transform functions, such as a scikit-learn transformer, e.g. MinMaxScaler(), StandardScaler(), PowerTransformer() |

For `var1` below, the variable will be transformed using standardisation. For `obj1` below, the objective will be forced to not undergo any transformation. For `obj2` below, the objective will be transformed using a non-native NEMO transformer, the scikit-learn PowerTransformer transformer.

In [ ]:
from sklearn.preprocessing import PowerTransformer

# Create the variable objects
var1 = ContinuousVariable(name="variable1", lower_bound=0.0, upper_bound=100.0, transformer="standardisation")
var2 = ContinuousVariable(name="variable2", lower_bound=0.0, upper_bound=100.0)
var_list = VariablesList([var1, var2])

# Create the objective objects
obj1 = RegressionObjective(
    name="objective1",
    obj_max_bool=True,
    lower_bound=0.0,
    upper_bound=100.0,
    transformer="none",
)
obj2 = RegressionObjective(
    name="objective2",
    obj_max_bool=False,
    lower_bound=0.0,
    upper_bound=100.0,
    transformer=PowerTransformer(),
)
obj_list = ObjectivesList([obj1, obj2])

In [ ]:
# Instantiate the sampler
sampler = LatinHyperCubeSampling()

In [ ]:
# Instantiate the acquisition function
acq_func = ExpectedImprovement(num_candidates=4)

In [ ]:
# Set up the optimisation instance
optimisation = Optimisation(var_list, obj_list, acq_func, sampler=sampler)

In [ ]:
# Start the optimisation using the convenient run function that will run for the specified number of iterations
# X and Y arrays represent a hypothetical initial dataset
optimisation_data = optimisation.run(X, Y, number_of_iterations=50)